In [1]:
import os
import tensorflow as tf

In [2]:
class_name = os.listdir('./dataset')
class_name

['instagram', 'owner', 'sticker', 'tiktok', 'twitter', 'youtube']

In [3]:
MODEL_PATH = os.path.join('..', 'word2vec', 'model-64-32k-100k')
MODEL_PATH

'..\\word2vec\\model-64-32k-100k'

In [4]:
vocab_ds = tf.data.TextLineDataset(os.path.join(MODEL_PATH, 'metadata.tsv')).filter(
        # ignore [UNK] token
        lambda text: tf.cast(not tf.strings.regex_full_match(text, '\[UNK\]'), bool))


In [5]:
# Define the vocabulary size and the number of words in a sequence.
vocab_size = 32000
sequence_length = 64

vectorize_layer = tf.keras.layers.TextVectorization(
        max_tokens=vocab_size,
        output_sequence_length=sequence_length,
        # add vocab
        vocabulary=tf.constant(
            [text.numpy() for text in vocab_ds]))

In [6]:
vocab = vectorize_layer.get_vocabulary()

In [19]:
model = tf.keras.models.load_model('pretrained')

In [20]:
example = [
    "ubah foto tersebut menjadi stiker",
    "download sebuah video dari youtube",
    "bisakah kamu mengubah foto di atas menjadi sticker?",
    "dapatkah saya berbicara dengan owner anda untuk meminta bantuan?",
    "download video dari instagram",
    "unduhkan saya sebuah video dari tiktok",
    "saya mempunyai link video twitter, download video twitter tersebut",
    "buat stiker",
    "halo bagaimana kabar kamu?",
    "makan ayam goreng"
]
example = tf.constant([vectorize_layer(text).numpy() for text in example])

predicted = model.predict(example)

for pred in predicted:
    index = predicted.tolist().index(pred.tolist())
    score = max(pred)
    higest_index = pred.tolist().index(score)
    print("Input:", " ".join([vocab[each] for each in example[index].numpy()]))
    print("Prediction:", pred)
    print("Predicted label:", class_name[higest_index], ":", score)
    print("")

1/1 [==============================] - 4s 4s/step
Input: ubah foto tersebut menjadi stiker                                                           
Prediction: [1.0709771e-04 1.2925623e-02 9.7308278e-01 8.7076845e-03 7.5087228e-05
 5.1017310e-03]
Predicted label: sticker : 0.9730828

Input: download sebuah video dari youtube                                                           
Prediction: [4.9528456e-03 6.3821142e-05 3.6024849e-04 7.8769969e-03 1.2307867e-04
 9.8662311e-01]
Predicted label: youtube : 0.9866231

Input: bisakah kamu mengubah foto di atas menjadi [UNK]                                                        
Prediction: [0.00121241 0.12463619 0.81722116 0.01944517 0.00110361 0.0363814 ]
Predicted label: sticker : 0.81722116

Input: [UNK] saya berbicara dengan owner anda untuk meminta bantuan                                                       
Prediction: [2.7026160e-04 9.9885869e-01 3.8257951e-04 2.7746297e-05 4.2576852e-04
 3.4977402e-05]
Predicted label: owner